In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualizations_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix
import pydrake.symbolic as sym
import time
from pydrake.all import CspaceFreeLine, CspaceFreeRegion
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.all import VerificationOption

from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid

In [4]:
import logging
drake_logger = logging.getLogger("drake")
drake_logger.setLevel(logging.DEBUG)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")
box_file = FindResourceOrThrow("drake/C_Iris_Examples/assets/shelves.sdf")


parser = Parser(plant)


models = []
models.append(Parser(plant, scene_graph).AddModelFromFile(model_file, model_name = "arm1"))
models.append(Parser(plant, scene_graph).AddModelFromFile(model_file, model_name = "arm2"))
models.append(Parser(plant, scene_graph).AddModelFromFile(box_file))

sp = 0.0
x_fac = 0.5
arm_offset = 0.25
locs = [
        [0,arm_offset,0], #first arm location
        [0,-arm_offset,0], # second arm location
        #box location
        [x_fac, 1.4*sp,0.4],
        [x_fac,-1.4*sp,0.4], 
        [-x_fac,-1.4*sp,0.4],
        [-x_fac,1.4,0.4], 
        [0.0 ,0 , 0.95], 
        [0.0 ,0 , -0.05]] 
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(),
                     RigidTransform(locs[idx]))
    idx +=1


plant.Finalize()


        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros_like(plant.GetPositionLowerLimits())

#compute limits in t-space
limits_t = []
for q in [plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), q_star))

do_viz = True

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram


DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile mechanism = RUNFILES_{MANIFEST_FILE,DIR}
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/14dof_iiwas_prm_certify.runfiles/drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf' (good)
DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/C_Iris_Examples/assets/shelves.sdf' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/14dof_iiwas_prm_certify.runfiles/drake/C_Iris_Examples/assets/shelves.sdf' (good)
D

In [6]:
visualizer.meshcat2.Delete("/prm")

In [7]:
# draw prm
import prm
from pydrake.all import Rgba

# limits = [np.array(t_low), np.array(q_high)]

visualizer.meshcat2.Delete("/prm")

def plot_prm(nodes, adjacency_list, width, color = Rgba(0.0, 0.0, 1, 1), prefix = ""):
    plt_idx = 0
    for node_idx in range(nodes.shape[0]):
        pos1 = np.append(nodes[node_idx, :],0)
        for edge_idx in range(len(adjacency_list[node_idx])): 
            pos2 = np.append(nodes[adjacency_list[node_idx][edge_idx], :],0)
            name = f"/{prefix}prm/rm/line{plt_idx}"
            visualizer.meshcat2.SetLine(name,  np.hstack([pos1[:,np.newaxis], pos2[:, np.newaxis]]),
                                 line_width = width, rgba = color)
            plt_idx +=1
            
plotting_fn_handle = partial(plot_prm, width = 0.01)

def collision(pos, col_func_handle):
    return col_func_handle(pos[0], pos[1])

def collision_bad(pos, col_func_handle):
    return 1-col_func_handle(pos[0], pos[1])

prm_col_fn_handle = partial(collision, col_func_handle = visualizer.col_func_handle_rational)
prm_col_fn_handle_bad = partial(collision_bad, col_func_handle = visualizer.col_func_handle_rational)

PRM = prm.PRM( 
            limits_t,
            num_points = 20,
            col_func_handle = prm_col_fn_handle,
            num_neighbours = 5, 
            dist_thresh = 10,
            num_col_checks = 10,
            max_it = 1e3,
            verbose = True,
            plotcallback = None
            )

# PRM_bad = prm.PRM( 
#             limits_t,
#             num_points = 2000,
#             col_func_handle = prm_col_fn_handle_bad,
#             num_neighbours = 5, 
#             dist_thresh = .5,
#             num_col_checks = 10,
#             verbose = True,
#             plotcallback = None
#             )

# PRM.add_start_end(start, target)
PRM.plot()
tot_num_edges = len(PRM.adjacency_list)* len(PRM.adjacency_list[0])

[PRM] Samples 0
[PRM] Nodes connected: 0


In [8]:
def make_prm_line_pairs(PRM):
    endpoint_index_set = set()
    for neighbors in PRM.adjacency_list:
        for n in neighbors[1:]:
            endpoint_index_set.add((neighbors[0], n))
    s0 = np.zeros((len(endpoint_index_set), plant.num_positions()))
    s1 = np.zeros((len(endpoint_index_set), plant.num_positions()))
    for i, (idx0, idx1) in enumerate(endpoint_index_set):
        s0[i] = PRM.nodes[idx0]
        s1[i] = PRM.nodes[idx1]
    return s0, s1
s0_mat_safe, s1_mat_safe = make_prm_line_pairs(PRM)
# s0_mat_unsafe, s1_mat_unsafe = make_prm_line_pairs(PRM_bad)
print(s0_mat_safe.shape)
# print(s0_mat_unsafe.shape)

(0, 14)


In [9]:
PRM.adjacency_list

[[0],
 [1],
 [2],
 [3],
 [4],
 [],
 [6],
 [7],
 [8],
 [9],
 [10],
 [11],
 [12],
 [13],
 [14],
 [15],
 [16],
 [17],
 [18],
 [19]]

## Set up the sliders so we can move the plant around manually

In [10]:
sliders = []
for i in range(plant.num_positions()):
    sliders.append(widgets.FloatSlider(min=plant.GetPositionLowerLimits()[i],
                                       max=plant.GetPositionUpperLimits()[i], 
                                       value=0, description=f'q{i}'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = np.zeros(plant.num_positions())
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.showres(q)
    visualizer.visualize_planes()
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

FloatSlider(value=0.0, description='q0', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='q1', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='q2', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='q3', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='q4', max=2.96706, min=-2.96706)

...

FloatSlider(value=0.0, description='q9', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='q10', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='q11', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='q12', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='q13', max=3.05433, min=-3.05433)

In [11]:
# filter fused joints self collisions so they don't interfere with collision engine
digaram = visualizer.diagram
context = visualizer.diagram_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(f"Number of pairs {len(pairs)}")


Number of pairs 147


In [12]:
def count_num_safe(safe_points_vect):
    num_safe = 0
    for val in safe_points_vect:
        if(val):
            num_safe += 1
    return num_safe
def make_certifier(use_affine = True, num_points = -1):
    t0 = time.time()
    plane_type = rational_forward_kinematics.SeparatingPlaneOrder.kAffine if use_affine \
                    else rational_forward_kinematics.SeparatingPlaneOrder.kConstant
    line_certifier = CspaceFreeLine(diagram, plant, scene_graph,
                                    plane_type, 
                                    q_star,
                                    set(), VerificationOption()
                                   )
    t1 = time.time()
    print(f"Time to construct line certifier = {t1-t0}s")
    return line_certifier

def do_certify(line_certifier, num_points = -1):
    t0 = time.time()
    safe_points, planes = line_certifier.CertifyTangentConfigurationSpaceLines(s0_mat_safe[:num_points,:], s1_mat_safe[:num_points,:])
    t1 = time.time()
    print(f"Certification of safe PRM in {t1-t0}s")
    num_safe1 = count_num_safe(safe_points)
    print(f"Certified {num_safe1}/{len(safe_points)} as safe")
    print(f"Failed to certify {len(safe_points)-num_safe1}/{len(safe_points)} safe points as safe")
    
    return (safe_points, planes, num_safe1)

In [ ]:
t0 = time.time()
line_certifier = CspaceFreeLine(diagram, plant, scene_graph,
                                rational_forward_kinematics.SeparatingPlaneOrder.kAffine, 
                                q_star,
                                set(), VerificationOption()
                               )
t1 = time.time()
print(f"Time to construct line certifier = {t1-t0}s")

DEBUG:drake:constructing link geometry pairs
DEBUG:drake:Generating Rationals from parent class
DEBUG:drake:doing separating planes 1/147
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:doing separating planes 2/147
DEBUG:drake:degree of polynomial = 2
DEBUG:drake:degree of polynomial = 2
DEBUG:drake:degree of polynomial = 2
DEBUG:drake:degree of polynomial = 2
DEBUG:drake:degree of polynomial = 2
DEBUG:drake:degree of polynomial = 2
DEBUG:

DEBUG:drake:degree of polynomial = 3
DEBUG:drake:degree of polynomial = 3
DEBUG:drake:degree of polynomial = 3
DEBUG:drake:degree of polynomial = 3
DEBUG:drake:doing separating planes 14/147
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:doing separating planes 15/147
DEBUG:drake:degree of polynomial = 2
DEBUG:drake:degree of polynomial = 2
DEBUG:drake:degree of polynomial = 2
DEBUG:drake:degree of polynomial = 2
DEBUG:drake:degree of poly

DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:doing separating planes 27/147
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 0
DEBUG:drake:degree of polynomial = 0
DEBUG:drake:degree of polynomial = 0
DEBUG:drake:degree of polynomial = 0
DEBUG:drake:degree of polynomial = 0
DEBUG:drake:degree of polynomial = 0
DEBUG:drake:degree of polynomial = 0
DEBUG:drake:degree of polynomial = 0
DEBUG:drake:doing separating planes 28/147
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of polynomial = 1
DEBUG:drake:degree of poly

DEBUG:drake:degree of polynomial = 3
DEBUG:drake:degree of polynomial = 3
DEBUG:drake:degree of polynomial = 3
DEBUG:drake:degree of polynomial = 3
DEBUG:drake:degree of polynomial = 3
DEBUG:drake:degree of polynomial = 3
DEBUG:drake:doing separating planes 40/147
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:doing separating planes 41/147
DEBUG:drake:degree of polynomial = 5
DEBUG:drake:degree of polynomial = 5
DEBUG:drake:degree of poly

DEBUG:drake:degree of polynomial = 3
DEBUG:drake:degree of polynomial = 3
DEBUG:drake:degree of polynomial = 3
DEBUG:drake:degree of polynomial = 3
DEBUG:drake:degree of polynomial = 3
DEBUG:drake:degree of polynomial = 3
DEBUG:drake:degree of polynomial = 3
DEBUG:drake:doing separating planes 53/147
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:degree of polynomial = 4
DEBUG:drake:doing separating planes 54/147
DEBUG:drake:degree of polynomial = 5
DEBUG:drake:degree of poly

...

INFO:drake:numerator poly constructed in 0.015 s. Has degree: 7
DEBUG:drake:Done rational 806/2352
INFO:drake:numerator poly constructed in 0.016 s. Has degree: 7
DEBUG:drake:Done rational 807/2352
INFO:drake:numerator poly constructed in 0.015 s. Has degree: 7
DEBUG:drake:Done rational 808/2352
INFO:drake:numerator poly constructed in 0.132 s. Has degree: 7
DEBUG:drake:Done rational 809/2352
INFO:drake:numerator poly constructed in 0.026 s. Has degree: 7
DEBUG:drake:Done rational 810/2352
INFO:drake:numerator poly constructed in 0.032 s. Has degree: 7
DEBUG:drake:Done rational 811/2352
INFO:drake:numerator poly constructed in 0.024 s. Has degree: 7
DEBUG:drake:Done rational 812/2352
INFO:drake:numerator poly constructed in 0.016 s. Has degree: 7
DEBUG:drake:Done rational 813/2352
INFO:drake:numerator poly constructed in 0.025 s. Has degree: 7
DEBUG:drake:Done rational 814/2352
INFO:drake:numerator poly constructed in 0.016 s. Has degree: 7
DEBUG:drake:Done rational 815/2352
INFO:drake

DEBUG:drake:Done rational 888/2352
INFO:drake:numerator poly constructed in 0.0 s. Has degree: 3
DEBUG:drake:Done rational 889/2352
INFO:drake:numerator poly constructed in 0.0 s. Has degree: 3
DEBUG:drake:Done rational 890/2352
INFO:drake:numerator poly constructed in 0.0 s. Has degree: 3
DEBUG:drake:Done rational 891/2352
INFO:drake:numerator poly constructed in 0.0 s. Has degree: 3
DEBUG:drake:Done rational 892/2352
INFO:drake:numerator poly constructed in 0.0 s. Has degree: 3
DEBUG:drake:Done rational 893/2352
INFO:drake:numerator poly constructed in 0.0 s. Has degree: 3
DEBUG:drake:Done rational 894/2352
INFO:drake:numerator poly constructed in 0.0 s. Has degree: 3
DEBUG:drake:Done rational 895/2352
INFO:drake:numerator poly constructed in 0.0 s. Has degree: 3
DEBUG:drake:Done rational 896/2352
INFO:drake:numerator poly constructed in 0.002 s. Has degree: 5
DEBUG:drake:Done rational 897/2352
INFO:drake:numerator poly constructed in 0.001 s. Has degree: 5
DEBUG:drake:Done rational 

DEBUG:drake:Done rational 972/2352
INFO:drake:numerator poly constructed in 0.001 s. Has degree: 5
DEBUG:drake:Done rational 973/2352
INFO:drake:numerator poly constructed in 0.002 s. Has degree: 5
DEBUG:drake:Done rational 974/2352
INFO:drake:numerator poly constructed in 0.004 s. Has degree: 5
DEBUG:drake:Done rational 975/2352
INFO:drake:numerator poly constructed in 0.003 s. Has degree: 5
DEBUG:drake:Done rational 976/2352
INFO:drake:numerator poly constructed in 0.152 s. Has degree: 7
DEBUG:drake:Done rational 977/2352
INFO:drake:numerator poly constructed in 0.014 s. Has degree: 7
DEBUG:drake:Done rational 978/2352
INFO:drake:numerator poly constructed in 0.027 s. Has degree: 7
DEBUG:drake:Done rational 979/2352
INFO:drake:numerator poly constructed in 0.017 s. Has degree: 7
DEBUG:drake:Done rational 980/2352
INFO:drake:numerator poly constructed in 0.021 s. Has degree: 7
DEBUG:drake:Done rational 981/2352
INFO:drake:numerator poly constructed in 0.014 s. Has degree: 7
DEBUG:drak

INFO:drake:numerator poly constructed in 0.247 s. Has degree: 9
DEBUG:drake:Done rational 1055/2352
INFO:drake:numerator poly constructed in 0.277 s. Has degree: 9
DEBUG:drake:Done rational 1056/2352
INFO:drake:numerator poly constructed in 43.036 s. Has degree: 11
DEBUG:drake:Done rational 1057/2352
INFO:drake:numerator poly constructed in 2.474 s. Has degree: 11
DEBUG:drake:Done rational 1058/2352
INFO:drake:numerator poly constructed in 2.494 s. Has degree: 11
DEBUG:drake:Done rational 1059/2352
INFO:drake:numerator poly constructed in 2.496 s. Has degree: 11
DEBUG:drake:Done rational 1060/2352
INFO:drake:numerator poly constructed in 2.448 s. Has degree: 11
DEBUG:drake:Done rational 1061/2352
INFO:drake:numerator poly constructed in 2.421 s. Has degree: 11
DEBUG:drake:Done rational 1062/2352
INFO:drake:numerator poly constructed in 2.417 s. Has degree: 11
DEBUG:drake:Done rational 1063/2352
INFO:drake:numerator poly constructed in 2.415 s. Has degree: 11
DEBUG:drake:Done rational 1

DEBUG:drake:Done rational 1136/2352
INFO:drake:numerator poly constructed in 0.0 s. Has degree: 3
DEBUG:drake:Done rational 1137/2352
INFO:drake:numerator poly constructed in 0.0 s. Has degree: 3
DEBUG:drake:Done rational 1138/2352
INFO:drake:numerator poly constructed in 0.0 s. Has degree: 3
DEBUG:drake:Done rational 1139/2352
INFO:drake:numerator poly constructed in 0.0 s. Has degree: 3
DEBUG:drake:Done rational 1140/2352
INFO:drake:numerator poly constructed in 0.0 s. Has degree: 3
DEBUG:drake:Done rational 1141/2352
INFO:drake:numerator poly constructed in 0.0 s. Has degree: 3
DEBUG:drake:Done rational 1142/2352
INFO:drake:numerator poly constructed in 0.0 s. Has degree: 3
DEBUG:drake:Done rational 1143/2352
INFO:drake:numerator poly constructed in 0.0 s. Has degree: 3
DEBUG:drake:Done rational 1144/2352
INFO:drake:numerator poly constructed in 0.0 s. Has degree: 3
DEBUG:drake:Done rational 1145/2352
INFO:drake:numerator poly constructed in 0.0 s. Has degree: 3
DEBUG:drake:Done rat

In [ ]:
i=0
t0 = time.time()
safe_points, planes = line_certifier.CertifyTangentConfigurationSpaceLine(s0_mat_safe[i,:], s0_mat_safe[i,:])
t1 = time.time()
print(f"Certification of safe PRM in {t1-t0}s")
print(f"Edge {i} is safe = {safe_points}")
print(f"Number of collision pairs is = {len(planes)}")